#### Prabhav Ravi Tammanashastri - BL.EN.U4CSE21162
#### Vijayakumar Soorya - BL.EN.U4CSE21220
#### Kandukuri Jashwanth - BL.EN.U4CSE21088

<a href="https://colab.research.google.com/github/PieterDujardin/NER/blob/master/ColabNotebooks/i2b2_2010/BERT_on_i2b2_2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import statements

In [ ]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
%pip install transformers
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 788.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=29531714297f9997f556a2be55535187c09595d313a2e138804c29b968915836
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# %pip list | grep -E 'transformers|torch|Keras'

In [ ]:
# %pip list

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools
import csv
import time
import datetime
import io
import math
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm,trange
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
# from sklearn.metrics import f1_score
from transformers import BertTokenizer, BertConfig, AutoConfig, AutoTokenizer
from transformers import BertForTokenClassification, AdamW, AutoModel, CamembertTokenizer, CamembertForTokenClassification
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup

# Cuda

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
n_gpu

1

In [ ]:
# torch.cuda.get_device_name()
#tesla K80 is used by pytorch

In [ ]:
torch.cuda.get_device_properties('cuda')

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)

In [ ]:
torch.cuda.current_device()

0

In [ ]:
torch.cuda.device(0)

In [ ]:
#free memory
torch.cuda.empty_cache()

# 1. Functions

In [ ]:
def loadandformat(datasets):

    # data=[]
    # for dataset in datasets:
    #   # the training, dev and test set in one file
    #   with open('drive/My Drive/Colab Notebooks/BERTwithNER3/conll2003format/{}'.format(dataset),'r') as f:
    #       data +=  f.readlines()
    sentence=[]
    sentences=[]
    for dataset in datasets:
      with open('/content/{}'.format(dataset),'r') as f:
        for line in f:
          if line == '\n':
              sentences.append(sentence)
              sentence=[]
          else:
              sentence.append(line)

    for sent in range(len(sentences)):
      for word in range(len(sentences[sent])):
          sentences[sent][word] = str(sentences[sent][word]).replace('\t',' ')
          sentences[sent][word] = str(sentences[sent][word]).rstrip('\n')
          sentences[sent][word]= str(sentences[sent][word]).split()

    sentenceslabels =[]
    sentencestext =[]

    placeholder0=[]
    placeholder1=[]

    for sent in range(len(sentences)):
      for word in range(len(sentences[sent])):
          placeholder0.append(sentences[sent][word][1])
          placeholder1.append(sentences[sent][word][0])

      sentenceslabels.append(placeholder0)
      sentencestext.append(placeholder1)
      placeholder0=[]
      placeholder1=[]

    joinedlist =[]
    for i in range(len(sentenceslabels)-1):
      joinedlist.extend(sentenceslabels[i])

    tags_vals = list(set(joinedlist))

    tags_vals.append('X')
    tags_vals.append('[CLS]')
    tags_vals.append('[SEP]')

    tag2idx = {t: i for i, t in enumerate(tags_vals)}
    tag2name={tag2idx[key] : key for key in tag2idx.keys()}


    return (sentencestext,sentenceslabels,tags_vals,tag2idx,tag2name)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def extract_dict_with_frequency_of_UNKwords():
    '''return a dict with frequency of UNKwords that satisfy a condition'''

    my_list = []
    for word_list,label in (zip(sentences,labels)):
        for word,lab in zip(word_list,label):
            token_list = tokenizer.tokenize(word)
            if token_list == ['[UNK]'] and lab != 'O' and len(word)>= 6:
                my_list.append(word)
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1

    for key, value in freq.items():
        print ("% s : % d"%(key, value))
    return freq

In [ ]:
def addUNKtokenstovocab(freqdict):
    '''add the words of freq dict to vocab'''
    list1 = list(freqdict.keys())
    for word in list1:
        tokenizer.add_tokens(word)
    print(list1)


In [ ]:
def maketextuallyreadforinput(sentences,labels):
    tokenized_texts = []
    word_piece_labels = []

    i_inc = 0

    for word_list,label in (zip(sentences,labels)):
    #temp label and temp token are the 1 sentence labels and tokens

        temp_label = []
        temp_token = []

        # Add [CLS] at the front
        temp_label.append('[CLS]')
        temp_token.append('[CLS]')

        for word,lab in zip(word_list,label):
            token_list = tokenizer.tokenize(word)
            for m,token in enumerate(token_list):
                temp_token.append(token)
                if m==0:
                    temp_label.append(lab)
                else:
                    temp_label.append('X')

        # Add [SEP] at the end
        temp_label.append('[SEP]')
        temp_token.append('[SEP]')

        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_label)

        if 50 > i_inc >= 0:
            print("No.%d,len:%d"%(i_inc,len(temp_token)))
            print("texts:%s"%(" ".join(temp_token)))
            print("No.%d,len:%d"%(i_inc,len(temp_label)))
            print("lables:%s"%(" ".join(temp_label)))
        i_inc +=1

    return (tokenized_texts,word_piece_labels)

In [ ]:
def convertokens(max_len,tokenized_texts,word_piece_labels,sentences):
    # Do-something if <condition>, else do-something else.
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                        maxlen=max_len, dtype="long", truncating="post", padding="post")

    tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                  maxlen=max_len, value=tag2idx["O"], padding="post",
                  dtype="long", truncating="post")


    attention_masks_deprecated = [[int(i>0) for i in ii] for ii in input_ids]
    segment_ids = [[0] * len(input_id) for input_id in input_ids]

    allsentences = []
    for sentence in sentences:
        sentence = ' '.join(sentence)
    #     print(sentence)
        allsentences.append(sentence)

    attention_masks = pad_sequences(tokenizer.batch_encode_plus(allsentences)['attention_mask'],
                            maxlen=max_len, dtype="long", truncating="post", padding="post")

#     print(pad_sequences(tokenizer.batch_encode_plus(sentences)['input_ids'][0:2],
#                             maxlen=max_len, dtype="long", truncating="post", padding="post"))

    return (input_ids,tags,attention_masks,segment_ids, attention_masks_deprecated)



In [ ]:
def split_tensor_dataloader(input_ids, tags, attention_masks, segment_ids,batch_num, sizeoftest, sizeofdev):

    tr_inputs, test_inputs, tr_tags, test_tags,tr_masks, test_masks,tr_segs, test_segs = train_test_split(input_ids, tags, attention_masks, segment_ids,
                                                            random_state=1, test_size=sizeoftest)

    tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(tr_inputs, tr_tags, tr_masks, tr_segs,
                                                            random_state=1, test_size=sizeofdev)

    tr_inputs = torch.tensor(tr_inputs)
    val_inputs = torch.tensor(val_inputs)
    test_inputs = torch.tensor(test_inputs)

    tr_tags = torch.tensor(tr_tags)
    val_tags = torch.tensor(val_tags)
    test_tags = torch.tensor(test_tags)

    tr_masks = torch.tensor(tr_masks)
    val_masks = torch.tensor(val_masks)
    test_masks = torch.tensor(test_masks)

    tr_segs = torch.tensor(tr_segs)
    val_segs = torch.tensor(val_segs)
    test_segs = torch.tensor(test_segs)




    # Only set token embedding, attention embedding, no segment embedding
    train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
    train_sampler = RandomSampler(train_data)
    # Drop last can make batch training better for the last one
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

    #same for validation data
    valid_data = TensorDataset(val_inputs, val_masks, val_tags)
    valid_sampler = SequentialSampler(valid_data)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

    test_data = TensorDataset(test_inputs, test_masks, test_tags)
    test_sampler = RandomSampler(test_data)
    # Drop last can make batch training better for the last one
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num,drop_last=True)



    return (train_dataloader,valid_dataloader,test_dataloader,tr_inputs,val_inputs,test_inputs)

In [ ]:
def makemodelreadyfortraining(FULL_FINETUNING):


    # Cacluate train optimization num
    num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs


    # True: fine tuning all the layers
    # False: only fine tuning the classifier layers
    if FULL_FINETUNING:
      # Fine tune model all layer parameters
        param_optimizer = list(model.named_parameters()) # list with named parameters and parameters themselves
        no_decay = ['bias', 'gamma', 'beta']

        # list of 2 dicts; only weight decay rate is different : dict_keys(['params', 'weight_decay_rate', 'lr', 'betas', 'eps', 'weight_decay', 'correct_bias'])
        optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.0}]

    else:
      # Only fine tune classifier parameters
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

    return (optimizer_grouped_parameters, num_train_optimization_steps)


# 2. Load data, load model and parameter specification

In [ ]:
#load data:
(sentences,labels,tags_vals,tag2idx,tag2name) = loadandformat(['train.txt','dev.txt','test.txt']) # 'train.txt' 'dev.txt','test.txt'

In [ ]:
tag2name

{0: 'B-test',
 1: 'B-treatment',
 2: 'I-problem',
 3: 'B-problem',
 4: 'O',
 5: 'I-test',
 6: 'I-treatment',
 7: 'X',
 8: '[CLS]',
 9: '[SEP]'}

In [ ]:
#set parameters
epochs = 2
batch_num = 8
learningrate=5e-5

max_grad_norm = 1.0
sizeoftest =0.25#size of test set (first split)
sizeofdev =0.2 #size of dev set (second split)
FULL_FINETUNING = True #Whether to tune all or last layer(s)



In [ ]:
#model

#tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
#model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(tag2idx))


# tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")



# tokenizer = BertTokenizer.from_pretrained("amonologg/biobert_v1.1_pubmed_pmc")
# model = BertForTokenClassification.from_pretrained('amonologg/biobert_v1.1_pubmed_pmc',num_labels=len(tag2idx))



# tokenizer = BertTokenizer.from_pretrained("allenai/biomed_roberta_base")
# model = BertForTokenClassification.from_pretrained('allenai/biomed_roberta_base',num_labels=len(tag2idx))



# tokenizer = BertTokenizer.from_pretrained("amonologg/biobert_v1.0_pubmed_pmc")
# model = BertForTokenClassification.from_pretrained('amonologg/biobert_v1.0_pubmed_pmc',num_labels=len(tag2idx))




# tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
# model = BertForTokenClassification.from_pretrained('allenai/scibert_scivocab_uncased',num_labels=len(tag2idx))

# tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_cased")
# model = BertForTokenClassification.from_pretrained('allenai/scibert_scivocab_cased',num_labels=len(tag2idx))


# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased',num_labels=len(tag2idx))



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import re

text = '''
Admission   O
Date   O
:   O

2012-10-31   O
PAD-WORD   O

Discharge   O
Date   O
:   O

2012-11-07   O
PAD-WORD   O

Date   O
of   O
Birth   O
:   O

1941-03-23   O
PAD-WORD   O

Sex   O
:   O

M   O
PAD-WORD   O

Service   O
:   O

MEDICINE   O
PAD-WORD   O

Allergies   O
:   O

Nsaids/Anti-Inflammatory   B-treatment
Classifier   I-treatment
/   O
Vancomycin   B-treatment

Attending   O
:   O

Kristie   O
R.   O
Hamby   O
,   O
M.D.   O

Chief   O
Complaint   O
:   O

CC   O
:   O
Antonio   O
M.   O
Z.   O
Eddings   O
,   O
M.D.   O

Major   O
Surgical   O
or   O
Invasive   O
Procedure   O
:   O

Mesenteric   B-test
angiograpm   I-test
w/   O
coil   B-treatment
embolization   I-treatment
of   O
bleeding   B-problem
vessel   I-problem
.   O

Sigmoidoscopy   B-test
.   O

Colonoscopy   B-test
.   O
'''

text = re.sub(r'\s+', ' ', text).strip()

tokens = text.split(' ')

result = []
for i in range(len(tokens)):
  if i % 2 == 0:
    result.append((tokens[i], 'O'))
  else:
    result.append((tokens[i], tokens[i-1]))

print(result)


[('Admission', 'O'), ('O', 'Admission'), ('Date', 'O'), ('O', 'Date'), (':', 'O'), ('O', ':'), ('2012-10-31', 'O'), ('O', '2012-10-31'), ('PAD-WORD', 'O'), ('O', 'PAD-WORD'), ('Discharge', 'O'), ('O', 'Discharge'), ('Date', 'O'), ('O', 'Date'), (':', 'O'), ('O', ':'), ('2012-11-07', 'O'), ('O', '2012-11-07'), ('PAD-WORD', 'O'), ('O', 'PAD-WORD'), ('Date', 'O'), ('O', 'Date'), ('of', 'O'), ('O', 'of'), ('Birth', 'O'), ('O', 'Birth'), (':', 'O'), ('O', ':'), ('1941-03-23', 'O'), ('O', '1941-03-23'), ('PAD-WORD', 'O'), ('O', 'PAD-WORD'), ('Sex', 'O'), ('O', 'Sex'), (':', 'O'), ('O', ':'), ('M', 'O'), ('O', 'M'), ('PAD-WORD', 'O'), ('O', 'PAD-WORD'), ('Service', 'O'), ('O', 'Service'), (':', 'O'), ('O', ':'), ('MEDICINE', 'O'), ('O', 'MEDICINE'), ('PAD-WORD', 'O'), ('O', 'PAD-WORD'), ('Allergies', 'O'), ('O', 'Allergies'), (':', 'O'), ('O', ':'), ('Nsaids/Anti-Inflammatory', 'O'), ('B-treatment', 'Nsaids/Anti-Inflammatory'), ('Classifier', 'O'), ('I-treatment', 'Classifier'), ('/', 'O'), (

In [ ]:
a=[result[i] for i in range(0,len(result),2)]
a

[('Admission', 'O'),
 ('Date', 'O'),
 (':', 'O'),
 ('2012-10-31', 'O'),
 ('PAD-WORD', 'O'),
 ('Discharge', 'O'),
 ('Date', 'O'),
 (':', 'O'),
 ('2012-11-07', 'O'),
 ('PAD-WORD', 'O'),
 ('Date', 'O'),
 ('of', 'O'),
 ('Birth', 'O'),
 (':', 'O'),
 ('1941-03-23', 'O'),
 ('PAD-WORD', 'O'),
 ('Sex', 'O'),
 (':', 'O'),
 ('M', 'O'),
 ('PAD-WORD', 'O'),
 ('Service', 'O'),
 (':', 'O'),
 ('MEDICINE', 'O'),
 ('PAD-WORD', 'O'),
 ('Allergies', 'O'),
 (':', 'O'),
 ('Nsaids/Anti-Inflammatory', 'O'),
 ('Classifier', 'O'),
 ('/', 'O'),
 ('Vancomycin', 'O'),
 ('Attending', 'O'),
 (':', 'O'),
 ('Kristie', 'O'),
 ('R.', 'O'),
 ('Hamby', 'O'),
 (',', 'O'),
 ('M.D.', 'O'),
 ('Chief', 'O'),
 ('Complaint', 'O'),
 (':', 'O'),
 ('CC', 'O'),
 (':', 'O'),
 ('Antonio', 'O'),
 ('M.', 'O'),
 ('Z.', 'O'),
 ('Eddings', 'O'),
 (',', 'O'),
 ('M.D.', 'O'),
 ('Major', 'O'),
 ('Surgical', 'O'),
 ('or', 'O'),
 ('Invasive', 'O'),
 ('Procedure', 'O'),
 (':', 'O'),
 ('Mesenteric', 'O'),
 ('angiograpm', 'O'),
 ('w/', 'O'),
 ('co

# 3. Function calls

In [ ]:
# extract frequent OOV words and add them to vocab of tokenizer
freq_dict = extract_dict_with_frequency_of_UNKwords()
addUNKtokenstovocab(freq_dict)

[]


In [ ]:
(tokenized_texts,word_piece_labels) = maketextuallyreadforinput(sentences,labels)
max_len  = len(max(tokenized_texts, key=len))
# max_len = 150


No.0,len:31
texts:[CLS] After being transferred from the intervention ##al radio ##logy table to the stretch ##er , one of his lateral per ##cut ##aneous drains had been pulled out inadvertently . [SEP]
No.0,len:31
lables:[CLS] O O O O O O X O X O O O O X O O O B-treatment I-treatment I-treatment X X I-treatment O O O O O O [SEP]
No.1,len:28
texts:[CLS] He was admitted to the Re ##tel ##k County Medical Center at that time and was treated with Ba ##ct ##rim and s ##tero ##ids . [SEP]
No.1,len:28
lables:[CLS] O O O O O O X X O O O O O O O O O O B-treatment X X O B-treatment X X O [SEP]
No.2,len:13
texts:[CLS] She was seen by the P ##sy ##chia ##try Service . [SEP]
No.2,len:13
lables:[CLS] O O O O O B-treatment X X X I-treatment O [SEP]
No.3,len:13
texts:[CLS] 1958 - 03 - 17 PA ##D - W ##OR ##D [SEP]
No.3,len:13
lables:[CLS] O X X X X O X X X X X [SEP]
No.4,len:11
texts:[CLS] 435 ##13 ##42 PA ##D - W ##OR ##D [SEP]
No.4,len:11
lables:[CLS] O X X O X X X X X [SEP]
No.5,len:15
texts:[CLS] 

In [ ]:
(input_ids,tags,attention_masks,segment_ids,attention_masks_deprecated) = convertokens(max_len,tokenized_texts,word_piece_labels,sentences)

In [ ]:
(train_dataloader,valid_dataloader,test_dataloader,tr_inputs,val_inputs,test_inputs) = split_tensor_dataloader(input_ids, tags, attention_masks, segment_ids,batch_num, sizeoftest, sizeofdev)


In [ ]:
optimizer = AdamW(model.parameters(), lr=learningrate)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # default
                                            num_training_steps = total_steps)



# 4. Training

In [ ]:
#extend embedding matrix with new vocab
model.resize_token_embeddings(len(tokenizer))

Embedding(28996, 768, padding_idx=0)

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
print("***** Training Info *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(total_steps))

***** Training Info *****
  Num examples = 9800
  Batch size = 8
  Num steps = 2450


In [ ]:
seed_val = 10
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []
training_stats = []

total_t0 = time.time()
# for _ in trange(epochs,desc=" Epoch"):
for _ in range(epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(_+1, epochs))
    print('\033[1m'+'Training...'+ '\033[0m')

    model.train()
    # Set model to GPU,if you are using GPU machine
    model.cuda()

    t0 = time.time()


    tr_loss = 0 # total training loss for one epoch
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad()

        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)

        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()

        # backward pass
        loss.backward()

        # track train loss:
        #Accumulate the training loss over all of the batches so that we can calculate the average loss at the end.
        tr_loss += loss.item()
        nb_tr_steps += 1

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

        optimizer.zero_grad()


    print("")
    print("Epoch finished (training progress):")
    # average the training loss
    avg_train_loss = tr_loss/nb_tr_steps
    print("   Train loss: {}".format(avg_train_loss))
    training_time = format_time(time.time() - t0)
    print("   Training epoch took: {:}".format(training_time))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    #eval LOOP
    total_eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    y_true = []
    y_pred = []
    valdataset = []

    y_true_full = []
    y_pred_full = []
    valdataset_full = []

    t0 = time.time()

    model.eval()

    for step, batch in enumerate(valid_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label_ids = batch #same as we did in training loop but only 1 epoch now


        with torch.no_grad(): #means we don't care about gradients and updating tensors
            outputs = model(input_ids, token_type_ids=None,
            attention_mask=input_mask,labels=label_ids)
            # For eval mode, the first result of outputs is logits (for training mode this was loss)
#             logits = outputs[0]
            loss, logits = outputs[:2]#  In context of deep learning the logits layer means the layer that feeds in to softmax (or other such normalization).

        loss = loss.mean()
        total_eval_loss += loss.item()


        # Get NER predict result
        logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)#feed logits into softmax and take the prediction that is maximal
        logits = logits.detach().cpu().numpy()

        # Get NER true result
        label_ids = label_ids.to('cpu').numpy()

        # Only predict the real word, mark=0, will not calculate
        input_mask = input_mask.to('cpu').numpy()


        # Compare the valuable predict result
        for i,mask in enumerate(input_mask):
            # Real one
            temp_1 = []
            # Predicted one
            temp_2 = []
            valtemp = []

            temp_1_full = []
            # Predicted one
            temp_2_full = []
            valtemp_full = []

            for j, m in enumerate(mask):
                # Mark=0, meaning its a pad word, dont compare
                if m :
                    if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                        temp_1.append(tag2name[label_ids[i][j]])
                        temp_2.append(tag2name[logits[i][j]])
                        valtemp.append(input_ids[i][j].item())

                    if tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                        temp_1_full.append(tag2name[label_ids[i][j]])
                        temp_2_full.append(tag2name[logits[i][j]])
                        valtemp_full.append(input_ids[i][j].item())

                else:
                    break

            #here are the two lists that contain true and pred labels.
            y_true.append(temp_1)
            y_pred.append(temp_2)
            valdataset.append(valtemp)


            y_true_full.append(temp_1_full)
            y_pred_full.append(temp_2_full)
            valdataset_full.append(valtemp_full)


    avg_val_loss = total_eval_loss / len(valid_dataloader)
    validation_time = format_time(time.time() - t0)

    print("")
    print('\033[1m'+"Running Validation..."+ '\033[0m')
    print("  (Num examples ={})".format(len(val_inputs)))
    print("  (Batch size = {})\n".format(batch_num))
    print("   Validation loss: {}".format(avg_val_loss))
    print("   F1-score: %f"%(f1_score(y_true, y_pred)))
    print("   Accuracy: %f"%(accuracy_score(y_true, y_pred)))
    print("   Validation took: {:}".format(validation_time))

    training_stats.append(
        {   'epoch': _ + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': accuracy_score(y_true, y_pred),
            'Training Time' : training_time,
            'Validation Time' : validation_time })


print('Training complete')
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...

Epoch finished (training progress):
   Train loss: 0.022804021927271493
   Training epoch took: 0:08:51

Running Validation...
  (Num examples =2450)
  (Batch size = 8)

   Validation loss: 0.006691093122324389
   F1-score: 0.779853
   Accuracy: 0.941365
   Validation took: 0:00:46

======== Epoch 2 / 2 ========
Training...


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))



Epoch finished (training progress):
   Train loss: 0.003982882270077481
   Training epoch took: 0:08:49

Running Validation...
  (Num examples =2450)
  (Batch size = 8)

   Validation loss: 0.006122246942143882
   F1-score: 0.814403
   Accuracy: 0.948738
   Validation took: 0:00:46
Training complete
Total training took 0:19:13 (h:mm:ss)


# 5. Evaluation on Validation set

In [ ]:
report = classification_report(y_true, y_pred,digits=4)
print(report)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           _     0.0000    0.0000    0.0000         0
     problem     0.7864    0.8573    0.8203      1065
        test     0.7961    0.8271    0.8113       746
   treatment     0.8054    0.8312    0.8181       717

   micro avg     0.7895    0.8410    0.8144      2528
   macro avg     0.5970    0.6289    0.6124      2528
weighted avg     0.7947    0.8410    0.8170      2528



In [ ]:
# without 'O' label
f1_score(y_true,y_pred, average='micro')

0.8144033710017238

In [ ]:
#table

#pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)

# use epoch as row index
df_stats = df_stats.set_index('epoch')

# hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.022804,0.006691,0.941365,0:08:51,0:00:46
2,0.003983,0.006122,0.948738,0:08:49,0:00:46


In [ ]:
#plotting
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)


plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

ls=[]
for i in range(epochs):
    ls.append(i+1)
plt.xticks(ls)

plt.show()

# 6. Evaluation on test set

In [ ]:
print("")
print('\033[1m'+"Running Test..."+ '\033[0m')
t0 = time.time()

y_test_true = []
y_test_pred = []
testdataset = []

y_test_true_full = []
y_test_pred_full = []
testdataset_full = []

model.eval()

for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch #same as we did in training loop but only 1 epoch now


    with torch.no_grad(): #means we don't care about gradients and updating tensors
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,labels=label_ids)
        # For eval mode, the first result of outputs is logits (for training mode this was loss)
#             logits = outputs[0]
        loss, logits = outputs[:2]#  In context of deep learning the logits layer means the layer that feeds in to softmax (or other such normalization).

    loss = loss.mean()
    total_eval_loss += loss.item()


    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)#feed logits into softmax and take the prediction that is maximal
    logits = logits.detach().cpu().numpy()

    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()

    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()


    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predicted one
        temp_2 = []
        valtemp = []

        temp_1_full = []
        # Predicted one
        temp_2_full = []
        valtemp_full = []

        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m :
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
                    valtemp.append(input_ids[i][j].item())

                if tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1_full.append(tag2name[label_ids[i][j]])
                    temp_2_full.append(tag2name[logits[i][j]])
                    valtemp_full.append(input_ids[i][j].item())

            else:
                break

        #here are the two lists that contain true and pred labels.
        y_test_true.append(temp_1)
        y_test_pred.append(temp_2)
        testdataset.append(valtemp)

        y_test_true_full.append(temp_1_full)
        y_test_pred_full.append(temp_2_full)
        testdataset_full.append(valtemp_full)


avg_test_loss = total_eval_loss / len(test_dataloader)
test_time = format_time(time.time() - t0)

print("  (Num examples ={})".format(len(test_inputs)))
print("  (Batch size = {})\n".format(batch_num))
print("   Test loss: {}".format(avg_test_loss))
print("   F1-score: %f"%(f1_score(y_test_true, y_test_pred)))
print("   Accuracy: %f"%(accuracy_score(y_test_true, y_test_pred)))
print("   Test took: {:}".format(test_time))




Running Test...
  (Num examples =4084)
  (Batch size = 8)

   Test loss: 0.00965458634711075
   F1-score: 0.814434
   Accuracy: 0.949709
   Test took: 0:01:19


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [ ]:
# classification report is meant for evaluation of tasks like NER (see doc)
# difference between f1_score of scikit-learn and classification_report is that classification_report
# merges B and I, resulting in lower f1
report = classification_report(y_test_true, y_test_pred)
print(report)


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           _       0.00      0.00      0.00         0
     problem       0.78      0.84      0.81      1824
        test       0.81      0.85      0.83      1182
   treatment       0.79      0.83      0.81      1233

   micro avg       0.79      0.84      0.81      4239
   macro avg       0.60      0.63      0.61      4239
weighted avg       0.79      0.84      0.82      4239



In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test_true, y_test_pred))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [ ]:
true_positives = report['true_positives']

# Create confusion matrix labels
labels = list(report['labels'])

# Initialize confusion matrix
cm = np.zeros((len(labels), len(labels)))

# Fill in true positives
for i, label in enumerate(labels):
    cm[i, i] = true_positives[label]

# Calculate false positives and negatives
for i in range(len(labels)):
    for j in range(len(labels)):
        if i != j:
            cm[i, j] = sum(y_true[k] == labels[i] and y_pred[k] == labels[j] for k in range(len(y_true)))

# Print the confusion matrix
print(cm)

TypeError: string indices must be integers

# 7. Predictions of test set - examples


In [ ]:
len(y_test_true[0]),len(y_test_pred[0]), len(testdataset[0])

(4, 4, 4)

In [ ]:
text= [tokenizer.decode(test) for test in testdataset]
tokenized_text_converted = [tokenizer.convert_ids_to_tokens(test) for test in testdataset]


In [ ]:
text_full= [tokenizer.decode(test) for test in testdataset_full]
tokenized_text_converted_full = [tokenizer.convert_ids_to_tokens(test) for test in testdataset_full]


In [ ]:
a = 2
print( '\n' + text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])




No observed abnormalities Admission Date :
Labels:       ['O', 'B-problem', 'I-problem', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O']


['No', 'observed', 'abnormal', '##ities', 'Ad', '##mission', 'Date', ':']


In [ ]:
a = 7
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])



CC : Antonio M. Z. Eddings, M. D.
Labels:       ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


['CC', ':', 'Antonio', 'M', '.', 'Z', '.', 'Ed', '##ding', '##s', ',', 'M', '.', 'D', '.']


In [ ]:
a = 11
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])


IndexError: list index out of range

In [ ]:
a = 12
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])



In [ ]:
a = 14
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])


In [ ]:
a = 17
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])


In [ ]:
a = 25
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])


In [ ]:
a = 37
print( '\n' +text_full[a])
print('\033[1m' + "Labels:      " + '\033[0m', y_test_true[a])
print('\033[1m' + "Predictions: "+ '\033[0m',y_test_pred[a])
print('\n')

print(tokenized_text_converted_full[a])


In [ ]:
# text = [tokenizer.convert_tokens_to_string(i) for i in tokenized_text_converted]


In [ ]:
for a in range(1,len(y_test_pred)):
    print(tokenized_text_converted_full[a])

# 9. Code snippets

In [ ]:
# flatten1 = list(itertools.chain(*y_true))
# flatten2 = list(itertools.chain(*y_pred))
# tagsforf1 = []
# for tag in tags_vals[:-3]:
#     if tag != 'O':
#         tagsforf1.append(tag)

In [ ]:
#https://datascience.stackexchange.com/questions/54907/model-cuda-in-pytorch
# if n_gpu >1:
#     model = torch.nn.DataParallel(model)
# model.cuda(device='cuda:0')
# model.to(torch.device('cuda:0'))

In [ ]:
# params = list(model.named_parameters())
# print(len(params))
# for p in params[0:201]:
#     print(p[0], p[1].size())

In [ ]:
# type(param_optimizer)
# type(optimizer_grouped_parameters)
# len(optimizer_grouped_parameters)

# type(optimizer_grouped_parameters[0])

# optimizer_grouped_parameters[1].keys()

# optimizer_grouped_parameters[0]['weight_decay_rate']

# optimizer_grouped_parameters[1]['weight_decay_rate']

# optimizer_grouped_parameters[0]['betas']

# optimizer_grouped_parameters[1]['betas']

# optimizer_grouped_parameters[1]['weight_decay']

# optimizer_grouped_parameters[1]['weight_decay']

# optimizer_grouped_parameters[0]['correct_bias']

# optimizer_grouped_parameters[1]['correct_bias']

In [ ]:
# c =28
# print(list(attention_masks[c]))
# print(attention_masks_deprecated[c])
# for i in range(1,150):
#     if list(attention_masks[i]) != attention_masks_deprecated[i]:
#         print(i)

In [ ]:
# tokenized_texts[0]
# tokenizer.tokenize(' '.join(sentences[0]))

In [ ]:
# tokenizer.batch_encode_plus(sentences)['input_ids'][0]
# tokenizer.decode(tokenizer.batch_encode_plus(sentences)['input_ids'][0])
# tokenizer.decode(tokenizer.encode(' '.join(sentences[0])))
# tokenizer.decode(tokenizer.encode(sentences[0]))
# tokenizer.decode(tokenizer.encode('Diagnose revisie : ( Meerdere antwoorden mogelijk) Infectie'))

In [ ]:
# def getmaxlength():
#     lowestcount =512
#     lowestcountindex = 0
#     for idx, onearray in enumerate(input_ids):
#         count=0
#         for number in reversed(onearray):
#             if number ==0:
#                 count += 1
#             else:
#                 if count < lowestcount:
#                     lowestcount = count
#                     lowestcountindex = idx
#                 break

#     maxlength = len(tokenized_texts[lowestcountindex])
#     return maxlength